### Prompt+LLM
#### 基本构成： PromptTemplate/ChatPromptTemplate  --> LLM/ChatModel  -->  OutPutParser

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi

prompt=ChatPromptTemplate.from_template("给我讲一个关于{foo}的笑话。")

llm=ChatTongyi(model="qwen-plus",temperature=0,max_tokens=1000)

In [ ]:
chain = prompt | llm
chain.invoke({"foo":"大灰狼"})

In [3]:
# 自定义停止输入符
chain = prompt | llm.bind(stop=["\n"])
chain.invoke({"foo":"北极熊"})

AIMessage(content='当然可以！这是一个关于北极熊的笑话：', additional_kwargs={}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': 'a0ce06d3-b155-9414-bd32-20462053f165', 'token_usage': {'input_tokens': 17, 'output_tokens': 10, 'total_tokens': 27, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--3321d0b4-f0e3-4cd3-964a-593fb3442a8b-0')

In [11]:
# 兼容openai函数调用的方式
functions=[
    {
        "name":"joke",
        "description":"讲笑话",
        "parameters":
        {
            "type":"object",
            "properties":
            {
                "setup":
                {
                    "type":"string",
                    "description":"笑话的开头"
                },
                "punchline":
                {
                    "type":"string",
                    "description":"爆梗的结尾"
                }
            },
            "required":["setup","punchline"]
        }
    }
]

chain = prompt | llm.bind(function_call={"name": "joke"}, functions=functions)

chain.invoke({"foo":"男人"},config={})

AIMessage(content='', additional_kwargs={}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': '0b08bcd9-ebfb-9ad6-8b78-436ef66c90b8', 'token_usage': {'input_tokens': 200, 'output_tokens': 34, 'total_tokens': 234, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--33d1a8a6-f6d8-468d-a97d-39e1291391f2-0')

In [12]:
# 输出解析器
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()

chain.invoke({"foo":"女人"})

'当然，这里有一个轻松幽默的小故事：\n\n一位女士去商店买鸡蛋。她拿起一盒鸡蛋仔细检查，然后放回去，再拿另一盒继续看。\n\n站在旁边的店员好奇地问：“女士，您在找什么吗？”\n\n女士回答：“我正在挑没有皱纹的鸡蛋。”\n\n希望这个小笑话能让你会心一笑！如果想要更多有趣的笑话，随时告诉我哦！'

In [14]:
# 与函数调用混合使用
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain=(
    prompt
    | llm.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

chain.invoke({"foo":"小人"})

OutputParserException: Could not parse function call: 'function_call'
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 